In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv('fault_dataset.csv')
data.head()

print(data.info())
print(data.describe())

from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder()
data['Fault Type'] = label_enc.fit_transform(data['Fault Type'])

X = data.drop(['Fault Type'], axis=1)
y = data['Fault Type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

conf_mat = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=label_enc.classes_, yticklabels=label_enc.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

import joblib
joblib.dump(model, 'fault_rf_model.pkl')

from ibm_watson_machine_learning import APIClient

wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "YOUR_API_KEY"
}

client = APIClient(wml_credentials)
client.set.default_project("YOUR_PROJECT_ID")

metadata = {
    client.repository.ModelMetaNames.NAME: "Fault Detection RF Model",
    client.repository.ModelMetaNames.TYPE: "scikit-learn_1.0",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_id_by_name("default_py3.10")
}

model_details = client.repository.store_model(model='fault_rf_model.pkl', meta_props=metadata)
model_id = model_details["metadata"]["id"]
print("Model ID:", model_id)

deployment = client.deployments.create(
    artifact_uid=model_id,
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: "Fault Detection Deployment",
        client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)

deployment_id = deployment["metadata"]["id"]
print("Deployment ID:", deployment_id)

scoring_payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields': X_test.columns.tolist(),
        'values': X_test.iloc[:5].values.tolist()
    }]
}

prediction = client.deployments.score(deployment_id, scoring_payload)
print("Prediction Result:\n", prediction)
